In [111]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import json

In [112]:
def get_movie_ids_from_list_page_soup(list_page_soup):
    """
    get movie_id from list_page_soup
    """
    movies = list_page_soup.find_all('div', class_="p-movie-cassette js-movie-cassette")
    
    if not movies:
        return False
    movie_ids = []
    
    for i in range(len(movies)):
        data_clip = movies[i].get('data-clip')
        movie_id = int(re.match(r'^.*"movie_id":(\d{5}).*', data_clip).group(1))
        movie_ids.append(movie_id)
        
    return movie_ids

In [113]:
def get_all_movie_ids(base_url):
    """
    get all movie ids from base url 
    (ex. https://filmarks.com/list/year/2010s/2018)
    """
    all_movie_ids = []
    page_number = 1
    while True :
        # request GET
        if page_number == 1:
            page = requests.get(base_url)
        else:
            payload = {'page': '{}'.format(page_number)}
            page = requests.get(base_url, params=payload)
        list_page_soup = BeautifulSoup(page.content, 'html.parser')
        
        movie_ids = get_movie_ids_from_list_page_soup(list_page_soup)
        if not movie_ids:
            break
        else:
            all_movie_ids.extend(movie_ids)
            page_number += 1
    return all_movie_ids

In [114]:
def get_rating_from_soup(soup):
    rating = soup.find('div', class_="c-rating__score").text
    try:
        return float(rating)
    except:
        return None
# get_rating_from_soup(get_soup_from_movie_id(85659))

In [115]:
def get_genres_from_soup(soup):
    try:
        lis = soup.find('div', class_="p-content-detail__genre").ul
        genres = [a.text for a in lis]
        return genres
    except:
        return None
# get_genres_from_soup(get_soup_from_movie_id(82210))

In [116]:
def get_synopsis_from_soup(soup):
    try:
        p = soup.find('div', {"id": "js-content-detail-synopsis"}) \
            .find("content-detail-synopsis") \
            .get(":outline") \
            .strip(r'"')
        #p = soup.find(class_="p-content-detail__synopsis-desc")
        return p
    except:
        return None
# get_synopsis_from_soup(get_soup_from_movie_id(82210))

In [117]:
def get_reviews_from_movie_id(movie_id):
    """
    get all reviews tied to a given movie_id
    and return a list of them
    """
    all_reviews = []
    base_url = 'https://filmarks.com/movies/' + str(movie_id)
    page_number = 1
    while True :
        # request GET
        if page_number == 1:
            page = requests.get(base_url)
        else:
            payload = {'page': '{}'.format(page_number)}
            page = requests.get(base_url, params=payload)
        individual_page_soup = BeautifulSoup(page.content, 'html.parser')
        
        reviews = get_reviews_from_individual_page_soup(individual_page_soup)
        if not reviews:
            break
        else:
            all_reviews.extend(reviews)
            print(len(reviews))
            page_number += 1
    return all_reviews

In [118]:
def get_soup_from_movie_id(movie_id):
    base_url = 'https://filmarks.com/movies/' + str(movie_id)
    page = requests.get(base_url)
    individual_page_soup = BeautifulSoup(page.content, 'html.parser')
    return individual_page_soup

In [124]:
def get_title_from_movie_id(movie_id):
    base_url = 'https://filmarks.com/movies/' + str(movie_id)
    page = requests.get(base_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_="p-content-detail__title").span.text
    
    return trans(title)

In [125]:
def trans(title_string):
    table = str.maketrans({
        '１': '1',
        '２': '2',
        '３': '3',
        '４': '4',
        '５': '5',
        '６': '6',
        '７': '7',
        '８': '8',
        '９': '9',
        '０': '0',
        '＆': '&',
        '％': '%',
         '＝': '=',
        '＄': '$',
        '＃': '#',
        '！': '!',
        '？': '?'
    })
    regulated_title = re.sub(r'（[^（）]*）', '', title_string)
    regulated_title = re.sub(r'\([^\(\)]*\)', '', regulated_title)
    regulated_title = re.sub(r'[\s\-～〜:：;、。<>＜＞「」\"\',\.・/／－]+', ' ', regulated_title)
    regulated_title = regulated_title.translate(table)
    regulated_title = regulated_title.rstrip(' ')
    return regulated_title


In [126]:
def write_json(title, genre, rating, synopsis):
    data = {}
    data["title"] = title
    data = {}
    data["genre"] = genre
    data["rating"] = rating
    data["synopsis"] = synopsis
    with open('jsons/metadata/{}.json'.format(title), "w", encoding="utf-8") as f:
        json.dump(data, f, sort_keys=True, indent=4, separators=(',', ': '))

In [127]:
def main(movie_ids):
    for movie_id in movie_ids:
        title = get_title_from_movie_id(movie_id)
        print("title: {}".format(title))
        
        soup = get_soup_from_movie_id(movie_id)
        genre = get_genres_from_soup(soup)
        rating = get_rating_from_soup(soup)
        synopsis = get_synopsis_from_soup(soup)
        print("genre: {}, rating: {}, synopsis: {}".format(genre, rating, synopsis))
        
        write_json(title, genre, rating, synopsis)
main(all_movie_ids)

title: 存在のない子供たち
genre: ['ドラマ'], rating: 4.3, synopsis: わずか12歳で、裁判を起こしたゼイン。訴えた相手は、自分の両親だ。裁判長から、「何の罪で?」と聞かれた ゼインは、まっすぐ前を見つめて「僕を産んだ罪」と答えた。中東の貧民窟に生まれたゼインは、両親が出生届を出さなかったために、自分の誕生日も知らないし、法的には社会に存在すらしていない。学校へ通うこともな く、兄妹たちと路上で物を売るなど、朝から晩まで両親に働かされている。唯一の支えだった大切な妹が 11 歳 で強制結婚させられ、怒りと悲しみから家を飛び出したゼインを待っていたのは、大人たちが作ったさらに過酷 な“現実”だった──。
title: メランコリック
genre: ['サスペンス', 'ドラマ', '青春'], rating: 3.9, synopsis: 名門大学を卒業後、うだつの上がらぬ生活を送っていた主人公・和彦。ある夜たまたま訪れた銭湯で高校の同級生・百合と出会ったのをきっかけに、その銭湯で働くこととなる。そして和彦は、その銭湯が閉店後の深夜、風呂場を「人を殺す場所」として貸し出していることを知る。そして同僚の松本は殺し屋であることが明らかになり…。 
title: 万引き家族
genre: ['ドラマ'], rating: 4.0, synopsis: 高層マンションの谷間にポツンと取り残された今にも壊れそうな平屋に、治と信代の夫婦、息子の祥太、信代の妹の亜紀の4人が転がり込んで暮らしている。彼らの目当ては、この家の持ち主である初枝の年金だ。足りない生活費は、万引きで稼いでいた。社会という海の底を這うような家族だが、なぜかいつも笑いが絶えず、互いに口は悪いが仲よく暮らしていた。\r\n冬のある日、近隣の団地の廊下で震えていた幼い女の子を、見かねた治が家に連れ帰る。体中傷だらけの彼女の境遇を思いやり、信代は娘として育てることにする。だが、ある事件をきっかけに家族はバラバラに引き裂かれ、それぞれが抱える秘密と切なる願いが次々と明らかになっていく──。
title: 永遠に僕のもの
genre: ['ドラマ', 'クライム'], rating: 3.7, synopsis: ブロンドの巻き毛に透き通る瞳、艶やかに濡れた瞳、磁器のように滑らかな

In [ ]:
import csv
#print(all_movie_ids)
with open('movie_ids.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(all_movie_ids)

In [ ]:
for i in len(all_movie_ids):
    movie_id = all_movie_ids[i]
    title = get_title_from_movie_id(movie_id)
    reviews = get_reviews_from_movie_id(movie_id)
    

In [23]:
type(all_movie_ids[0])

str

In [107]:
# read movie_ids.csv
import csv
all_movie_ids = []
with open('movie_ids.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        all_movie_ids.extend(row)

## TEST

In [ ]:
get_title_from_movie_id(79340)

In [20]:
write_json("おはよ", [1, 2, {'3': 3}, {'4': 4}])